In [13]:
import pandas_datareader as pd2
import pandas as pd
import tensorflow
import keras as ks
import matplotlib.pyplot as mp
from keras.layers import Dense,LSTM,Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping
import numpy as np
from ipywidgets import interact,interact_manual

In [9]:
#WE DECLARE OUR CLASSES
#THIS FUNCTION GETS, WINDOWS AND SPLITS INTO TEST/TRAIN
class Data():
    
    def __init__(self,ticker,start,end):
        self.data = pd2.DataReader(ticker,'yahoo', start, end).iloc[:,]
        
    def window(self,windows,col):
        
        #we sort the windowed array
        windows = sorted(windows)

        #we loop through the training data and restructure
        for i in range(max(windows),self.data.count()[0]):

            y_record = self.data.iloc[[i],:].loc[:,[col]]
            x_record = self.data.iloc[[i-windows[0]],:].loc[:,[col]]
            x_record.columns = str(windows[0]) + "-" + x_record.columns

            if(len(windows)>1):

                for j in range(1,len(windows)):
                    x_temp   = self.data.iloc[[i-windows[j]],:].loc[:,[col]]
                    x_temp.columns = str(windows[j]) + "-" + x_temp.columns
                    x_record = pd.concat([x_record.reset_index(drop=True), x_temp.reset_index(drop=True)], axis=1)

            if(i==max(windows)):
                x_final = x_record
                y_final = y_record
            else:
                x_final = x_final.append(x_record)
                y_final = y_final.append(y_record)

            self.features = x_final.set_index(y_final.index)
            self.response = y_final
            
    def split_data(self,period):
        self.test_features = self.features.iloc[self.features.count()[0]-period:self.features.count()[0],:]
        self.train_features = self.features.iloc[0:self.features.count()[0]-period,:]
        self.test_response = self.response.iloc[self.features.count()[0]-period:self.features.count()[0],:]
        self.train_response = self.response.iloc[0:self.features.count()[0]-period,:]
        self.data_train = self.data.iloc[0:self.data.count()[0]-period,:]
        self.data_test =  self.data.iloc[self.data.count()[0]-period:self.data.count()[0],:]

class NN():
    
    def __init__(self,neurons,activations):
        self.neurons = neurons
        self.activations = activations
     
    def create_model(self,dim,loss,optimizer):
    
        self.model = ks.Sequential()
     
        for i in range(0,len(self.neurons)):
            neuron = self.neurons[i]
            activation = self.activations[i]
        
            if(i == 0):
                self.model.add(Dense(neuron,input_dim = dim,activation = activation))
            else:
                self.model.add(Dense(neuron,activation = activation))
                
        self.model.compile(loss=loss,optimizer=optimizer)

In [5]:
a = False

def test():
    
    global a
    
    if a == True:
        print('Go1')
        a = False
    else:
        print('Go2')
        a = True
    

interact_manual(test)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function __main__.test()>

In [14]:
def test2():
    data = Data('AAPL','2012-05-05','2013-05-5')
    line(data.data.loc[:,'Open'])

interact_manual(test2)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function __main__.test2()>